## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Étiquetage morphosyntaxique (POS Tagging) 
- Lemmatisation
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données


**Précisions sur la structuration des différents objets créés (corpus, sents, tokens, ngrammes, tagged)**  
| Index dans la structure      | Information représentée  |
| ----------- | ----------- |
| `x[0]`      | 1 document dans le corpus (`corpus`)       |
| `x[0][0]`   | 1 phrase dans un document du corpus (`sents`) |
| `x[0][0][0]`    | 1 ngramme dans un document (`tokens` / `ngrammes`)        |
| `x[0][0][0]`   | 1 tuple (ngramme, patron syntaxique) dans un ngramme (`tuples`) |

**2022-06-17**  
  
*Modifications apportées*
- Traiter un ensemble de documents .txt plutôt qu'un seul gros corpus ✓
- Retenir seulement les termes qui ont une fréquence supérieure à x ✓
- Rammener l'étiquetage morphosyntaxique et le filtrage par patrons syntaxiques plus tôt afin de filtrer les KWIC ✓
  
**2022-06-22**  
  
*Modifications apportées* 
- Ajouter le tag associé au sous-corpus dans les paths (acteur + path) - je suis pas certaine si j'avais fini, s'il y a une erreur revoir si sous_corpus = False ou s'il faudrait ajouter un if
- Convertir les tags de TreeTagger selon les tags utilisés par le Lefff pour les fournir au lemmatiseur (performance meilleure lorsqu'on lui indique la POS)

*Modifications à ajouter* 
- Modifier le lemmatiseur pour lui fournir les POS tags et qu'il performe mieux

### **Lire le corpus** 

In [1]:
import os, shutil
from pathlib import Path

path = '/Users/camilledemers/Documents/03-corpus/2-sous-corpus/'
acteur = 'asso_ordres'
sous_corpus = True # Faut peut-être revoir le script pour adapter en fonction de ça (je me rappelle plus si j'avais fini)
tag = 'Nursing'

# Change the directory
os.chdir(path + acteur + '/' + tag + "/")
len(os.listdir())

4445

In [2]:
corpus = []

for file in os.listdir():
    if file.endswith(".txt") and not file.endswith('-corpus_FR.txt') and not 'PDF' in file:
        file_path = path + acteur + '/' +  tag + "/" + file
        
        try:
            with open(file_path, 'r', encoding = "UTF-8") as f:
                data = f.readlines()
                corpus.append(data[1].strip('\n').lower())

        except:
            print('Ce fichier-là n\'a pas pu être lu : ' + file)

In [3]:
corpus = corpus[:round(len(corpus))]

nb_docs = len(corpus)

print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 4443 documents.


### **Segmentation** (phrases)

**NLTK**\
https://www.nltk.org/ 

In [4]:
import nltk
#nltk.download(['popular'])

In [5]:
from nltk import sent_tokenize 

sents = [[s.strip('.') for s in sent_tokenize(doc)] for doc in corpus]

### **Tokenisation** (mots)

In [6]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w+")

In [7]:
tokens = [[tokenizer_re.tokenize(s) for s in doc] for doc in sents]

len_corpus = len(nltk.flatten(tokens))

In [8]:
print("Notre corpus contient {} tokens.".format(len_corpus))

Notre corpus contient 2905985 tokens.


### **Filtrage** (antidictionnaire)

In [9]:
# Importer l'antidictionnaire pour filtrer les données
from pandas import *

# Stopwords fréquents en français
path = "/Users/camilledemers/Documents/04-filtrage/stopwords.csv"
with open(path, 'r', encoding="utf-8") as f:
    stopwords = read_csv(f)
    stopwords = [t.lower() for t in stopwords['Stopwords'].tolist()]


# Stopwords fréquents en anglais
path = '/Users/camilledemers/Documents/04-filtrage/stop_words_english.txt'
with open(path, 'r', encoding="utf-8") as f:
    sw = [w.strip('\n').lower() for w in f.readlines()]

stopwords += sw

# Signes de ponctuation
import string 
punct = [s for s in string.punctuation] 
punct += ['»' ,'©', '']

stopwords += punct


# Mis en commentaire pour l'instant car ça allonge le délai de traitement
#Prénoms (curieusement, il y en a beaucoup dans les données)
# path = '/Users/camilledemers/Documents/04-filtrage/Prenoms.csv'
# with open(path, 'r', encoding='utf-8') as f:
#     sw = read_csv(f)
#     sw = [str(t).lower() for t in sw['01_prenom'].tolist()]

# stopwords += sw

#Noms de famille 
# path = '/Users/camilledemers/Documents/04-filtrage/nomsFamille.csv'
# with open(path, 'r', encoding='utf-8') as f:
#     sw = read_csv(f)
#     sw = [str(t).lower() for t in sw['Nom'].tolist()]

# stopwords += sw


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [10]:
path = '/Users/camilledemers/Documents/04-filtrage/mwe_stopwords.txt'

with open (path, 'r', encoding='utf-8') as f:
    mwe_sw = [tuple(tokenizer_re.tokenize(t)) for t in f.readlines()]
    #mwe_sw = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [11]:
from nltk.tokenize import MWETokenizer
tokenizer_mwe = MWETokenizer(mwe_sw, separator=' ')

In [12]:
mwe_sw = [tokenizer_mwe.tokenize(w)[0] for w in mwe_sw]

In [13]:
tokens = [[[t for t in tokenizer_mwe.tokenize(sent) if t not in mwe_sw] for sent in doc] for doc in tokens]

### **Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook

In [14]:
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.probability import FreqDist

In [15]:
from nltk.util import everygrams
ngrammes = [[list(everygrams(sent, min_len=2, max_len=4)) for sent in doc] for doc in tokens]

### **Filtrage (N-grammes)**

On retire les n-grammes qui n'apparaissent qu'une seule fois dans le corpus ou qui débutent ou terminent par :
- un stopword
- un mot de 1 lettre ou moins

Pour le reste du traitement, on arrête de considérer les frontières entre les phrases et entre les documents (nos ngrammes les respectent donc on n'en a plus besoin)

In [16]:
# Calculer la distribution de fréquence de chaque ngramme dans tout le corpus
# Pour ça, on va 'applatir' la liste des ngrammes pour ne plus tenir compte des frontières entre les phrases et entre les documents
def ng_flat(ngramme):
    liste    = [] 
    for doc in ngrammes:
        for sent in doc:
            for ngram in sent:
                liste.append(ngram)
    return liste

ngrammes = ng_flat(ngrammes)
freq = FreqDist(ngrammes)

In [17]:
len(ngrammes)

7750458

In [18]:
import pandas as pd, re

ngrammes = [ngram for ngram in ngrammes if freq[ngram] > 30 and \
        not ngram[0] in stopwords and len(ngram[0]) > 1 and not re.search('\d+', ngram[0])\
        and not ngram[-1] in stopwords and len(ngram[-1]) > 1]

In [19]:
len(ngrammes)

728174

In [20]:
phrases = [" ".join(ngram) for ngram in ngrammes]
freq = FreqDist(phrases)

In [21]:
vocabulaire = freq.keys()

def tabCSV(tab, titre):
    tab = pd.DataFrame(tab.items(), columns= ["Expression", "Fréquence"])
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/'

    if sous_corpus:
        path += tag + '/'
        file_path = path + acteur + '_' + tag
        
    
    else:
        file_path = path 

    Path(path).mkdir(parents=True, exist_ok=True)

    
    tab.to_csv(path + tag + titre + '.csv')

In [22]:

path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/' + tag + '/'

tabCSV(freq, '_n-grams')

### **POS Tagging** (TreeTagger)
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [23]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/Users/camilledemers/opt/anaconda3/lib/python3.9/site-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


### **Mapping POS Tags** (FRMG)

Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [24]:
path = '/Users/camilledemers/Documents/04-filtrage/mapping_treeTagger_lefff.csv'

with open(path) as f:
    csv = read_csv(f)

treeTag = [term for term in csv['TreeTagger'].tolist()] 
lefff = [term for term in csv['Lefff'].tolist()]

mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [25]:
ngrammes = [" ".join(ngramme) for ngramme in ngrammes]

In [26]:
tagged_extrant = [[ngram, " ".join([mapping[t.split('\t')[1]] for t in tagger.tag_text(ngram)])] for ngram in ngrammes]

In [48]:
tagged = [[term[0].split(), term[1]] for term in tagged_extrant]

In [51]:
test = tagged[0]

### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

**Reprendre ici pour arranger le lemmatiseur**

In [27]:
path = '/Users/camilledemers/Documents/04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:50] # Pour prendre seulement les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [28]:
terms = [t for t in tagged if t[1] in patterns]

In [29]:
path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/' + tag + '/'
tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique"])
tab = pd.DataFrame(tab.groupby(["Expression", "Structure syntaxique"]).size().reset_index(name="Fréquence"))
tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

                    
tab.to_csv(path + tag + '_phrases.csv')

### **Lemmatisation** (FrenchLefffLemmatizer)

https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

In [30]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [31]:
lemmatizer = FrenchLefffLemmatizer()

In [36]:
terms[0]

['aide financière', 'nc adj']

In [39]:
lemmas = [[term[0].split(), term[1].split()] for term in terms]

In [41]:
lemmas[0]

[['aide', 'financière'], ['nc', 'adj']]

In [ ]:
test = tuples[0]
test = [t.split() for t in test]
test test = []


In [ ]:
#path = '/Users/camilledemers/Documents/04-filtrage/' + tag + '/'
#tab = pd.DataFrame(tuples_lemmes, columns= ["Expression", "Variante", "Structure syntaxique"])
#tab = pd.DataFrame(tab.groupby(["Expression", "Structure syntaxique"]).size().reset_index(name="Fréquence"))
#tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
#tab.to_csv(path + tag + '_phrases_lemmatized-TreeTagger.csv')

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières 

kw = ['programme', 'plan', 'service', 'intervenant', 'institut', 'groupe de recherche', 'personne']

In [ ]:
phrases = [" ".join(ngram) for ngram in ngrammes]
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in phrases: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])

path = '/Users/camilledemers/Documents/04-filtrage' + '/' + tag + '/'
extrant.to_csv(path + tag + '_KWIC' +'.csv')

### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
path = '/Users/camilledemers/Documents/04-filtrage/MeSH/mesh-fr.txt'

with open (path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
extr_mesh = tokenizer_mesh.tokenize(terms)

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [ ]:
termes_mesh = FreqDist(termes_mesh)
tabCSV(termes_mesh, '_MeSH')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
path = '/Users/camilledemers/Documents/04-filtrage/SNOMED/SNOMED_fr.csv'

with open(path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize(terms)

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
termes_sm = FreqDist(termes_sm)
tabCSV(termes_sm, '_SNOMED')

### **Lemmatisation**

In [ ]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

In [ ]:
tagger.tag_text('spéciaux')

In [ ]:
lemmatizer.lemmatize('spéciaux')

In [ ]:
lemmatizer.lemmatize('spéciaux', 'adj') # Il faut aller mapper les étiquettes du Tree Tagger avec ceux que reçoit le lemmatiseur pour améliorer sa performance

**Tagset**  
http://alpage.inria.fr/frmgwiki/content/tagset-frmg
- 'adj'| Adjectif
- 'n'
- 'v'
- 'det'
- 'adv'
- 'prep'
- 'pro'
- 'np' | Nom propre

### **Filtrage (fréquence)**
Pour la suite du traitement, on ne retient que les N expressions les plus fréquentes dans le corpus

In [ ]:
freq_terms = FreqDist(terms)
exp_freq = freq_terms.most_common()

path = '/Users/camilledemers/Documents/05-transformation/' + tag + '/'
Path(path).mkdir(parents=True, exist_ok=True)

file_path = path + tag + '_terms.csv'

In [ ]:
tab = pd.DataFrame([t[0] for t in exp_freq], columns=["Terme"])
tab.to_csv(file_path)

print("Fini pour le corpus suivant : {}".format(tag))